In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
from typing import Annotated, Literal, TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import BaseMessage, HumanMessage

class ChatMessages(TypedDict):
    messages: list[BaseMessage]

In [13]:
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

In [14]:
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."

In [15]:
tools=[get_weather]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

In [16]:
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}


def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [17]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools)


In [18]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", 'agent')

graph = workflow.compile(checkpointer=MemorySaver())

In [19]:
graph.invoke(
    {"messages": [HumanMessage(content="How is the weather in munich?")]},
    config={"configurable": {"thread_id": 1}}
)

{'messages': [HumanMessage(content='How is the weather in munich?', additional_kwargs={}, response_metadata={}, id='5bc74b30-e751-4d74-81f5-e7335afa7526'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HhOCV9urcI6jvqQ1ntRFbquN', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 52, 'total_tokens': 67, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b53425c0-c354-4626-be49-966eb212406c-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_HhOCV9urcI6jvqQ1ntRFbquN', 'type': 'tool_call'}], usage_metadata={'input_tokens': 52, 'output_tokens': 15, 'total_tokens': 67,

In [20]:
graph.invoke(
    {"messages": [HumanMessage(content="What would you recommend to do in that city than?")]},
    config={"configurable": {"thread_id": 1}}
)

{'messages': [HumanMessage(content='How is the weather in munich?', additional_kwargs={}, response_metadata={}, id='5bc74b30-e751-4d74-81f5-e7335afa7526'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HhOCV9urcI6jvqQ1ntRFbquN', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 52, 'total_tokens': 67, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b53425c0-c354-4626-be49-966eb212406c-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_HhOCV9urcI6jvqQ1ntRFbquN', 'type': 'tool_call'}], usage_metadata={'input_tokens': 52, 'output_tokens': 15, 'total_tokens': 67,

### Getting production ready - async and streaming

In [21]:
model = ChatOpenAI(model="gpt-4o-mini", streaming=True).bind_tools(tools)

In [22]:
async def call_model(state: MessagesState):
    messages = state["messages"]
    response = await model.ainvoke(messages)
    return {"messages": [response]}

In [27]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", END: END},
)
workflow.add_edge("tools", "agent")

graph = workflow.compile(checkpointer=MemorySaver())


In [30]:
inputs = {"messages": [HumanMessage(content="How is the weather in Munich?")]}
config={"configurable": {"thread_id": 1}}

In [50]:
inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
async for output in graph.astream(inputs, stream_mode="updates", config=config):
    # stream_mode="updates" yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value["messages"][-1].pretty_print())

Output from node 'agent':
---
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_ySHrDuIZ6637GLZ2iMrdw7Ze)
 Call ID: call_ySHrDuIZ6637GLZ2iMrdw7Ze
  Args:
    location: San Francisco
None
Output from node 'tools':
---
================================= Tool Message =================================
Name: get_weather

It's 32 degrees Celsius and sunny.
None
Output from node 'agent':
---
================================== Ai Message ==================================

The weather in San Francisco is currently 32 degrees Celsius and sunny.
None


In [64]:
from langchain_core.messages import AIMessageChunk, HumanMessage

inputs = [HumanMessage(content="what is the weather in sf")]
gathered = None

async for msg, metadata in graph.astream({"messages": inputs}, stream_mode="messages", config=config):
    if msg.content and not isinstance(msg, HumanMessage):
        # Print each token as it streams in
        print(msg.content, end="|", flush=True)

    # Handle the AI message chunks for proper assembly
    if isinstance(msg, AIMessageChunk):
        if gathered is None:
            gathered = msg
        else:
            gathered = gathered + msg


It's 32 degrees Celsius and sunny.|The| weather| in| San| Francisco| is| currently| |32| degrees| Celsius| and| sunny|.|

In [66]:
print(gathered.content)

The weather in San Francisco is currently 32 degrees Celsius and sunny.
